# Face Detection with YOLOv5

In this jupyter notebook we'll be implementing face detection with ultralytics' YOLOv5 implementation using the PyTorch deep learning framework.

## Table of Contents

1. [Dependency Acquisition](##Dependency-Acquisition)
   1. [Dependency Installation](###Dependency-Installation)
   2. [Dependency Import](###Dependency-Import)
2. [Dataset Collection](##Dataset-Collection)
   1. [Image Capture](###Image-Capture)
   2. [Image Labelling](###Image-Labelling)
3. [Model Re-training](##Model-Re-training)
4. [Inference](##Inference)

## Dependency Acquisition

### Dependency Installation

PyTorch installation

In [2]:
!pip3 install torch torchvision torchaudio

You should consider upgrading via the 'c:\users\fouadhanani\virtualenvs\deep-learning\scripts\python.exe -m pip install --upgrade pip' command.


Ultralytics' YOLOv5 github repo clone and dependency installation

In [4]:
print("cloning github repo...")
!git clone https://github.com/ultralytics/yolov5
print("installing YOLOv5 requirements...")
!cd yolov5 & pip install -r requirements.txt

cloning github repo...
installing YOLOv5 requirements...


fatal: destination path 'yolov5' already exists and is not an empty directory.


You should consider upgrading via the 'c:\users\fouadhanani\virtualenvs\deep-learning\scripts\python.exe -m pip install --upgrade pip' command.


LabelImg (labelling tool) installation

In [5]:
print("cloning github repo...")
!git clone https://github.com/tzutalin/labelImg
print("Installing LabelImg requirements & finalizing installation...")
!pip install pyqt5 lxml --upgrade
!cd labelImg && pyrcc5 -o libs/resources.py resources.qrc

cloning github repo...
Installing LabelImg requirements & finalizing installation...


Cloning into 'labelImg'...


You should consider upgrading via the 'c:\users\fouadhanani\virtualenvs\deep-learning\scripts\python.exe -m pip install --upgrade pip' command.


### Dependency Import

In [63]:
import torch
import cv2
import uuid
import os
import numpy as np

## Dataset Collection

### Image Capture

Before we start capturing images using our camera, let's create some directories.

In [3]:
dataset_images_path = os.path.join("dataset", "images")
if not os.path.exists(dataset_images_path):
    os.makedirs(dataset_images_path)
    print(f"{dataset_images_path} created successfully...")

In [4]:
dataset_labels_path = os.path.join("dataset", "labels")
if not os.path.exists(dataset_labels_path):
    os.makedirs(dataset_labels_path)
    print(f"{dataset_labels_path} created successfully...")

Now let's start capturing some images.

In [57]:
capture = cv2.VideoCapture(0)

face, noface = 0, 0
label = "face"

while True:
    status, frame = capture.read()

    cv2.imshow("frame", frame)

    key = cv2.waitKey(1)
    # if the "f" key is pressed then switch the image label (face <--> noface)
    if key & 0xFF == ord('f'):
        label = "face" if label == "noface" else "noface"
        print(f"switching to the label {label}")

    # if the "s" key is pressed then save image
    if key & 0xFF == ord('s'):
        image_path = os.path.join(dataset_images_path, f"{label}-{uuid.uuid1()}.jpg")
        cv2.imwrite(image_path, frame)
        print(f"saving image of label {label} to {image_path}")
        if label == "face":
            face += 1
        else:
            noface += 1
        print(f"so far there is {face} images of face and {noface} images of noface")

    # if the "q" key is pressed then stop
    if key & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

saving image of label face to dataset\images\face-e04fd1e5-1d5f-11ed-928e-a24242e2012e.jpg
so far there is 1 images of face and 0 images of noface
saving image of label face to dataset\images\face-e28f9d3f-1d5f-11ed-892e-a24242e2012e.jpg
so far there is 2 images of face and 0 images of noface
saving image of label face to dataset\images\face-e3a37dde-1d5f-11ed-9d42-a24242e2012e.jpg
so far there is 3 images of face and 0 images of noface
saving image of label face to dataset\images\face-e52a0995-1d5f-11ed-bb55-a24242e2012e.jpg
so far there is 4 images of face and 0 images of noface
saving image of label face to dataset\images\face-e63bb925-1d5f-11ed-8c0e-a24242e2012e.jpg
so far there is 5 images of face and 0 images of noface
saving image of label face to dataset\images\face-e7440156-1d5f-11ed-82d5-a24242e2012e.jpg
so far there is 6 images of face and 0 images of noface
saving image of label face to dataset\images\face-e891bf51-1d5f-11ed-b0c0-a24242e2012e.jpg
so far there is 7 images of

### Image Labelling

We'll be using LabelImg to label our dataset of images, the following commands will run the labelImg software.

In [32]:
!cd labelImg & python labelImg.py # it's better to run in a terminal (copy everything and leave out the "!" and make sure you're connected to the right kernel)

usage: labelImg.py [-h] [image_dir] [class_file] [save_dir]
labelImg.py: error: unrecognized arguments: to run in a terminal (copy everything and leave out the ! and make sure you're connected to the right kernel)


The previous labelling action only applies to the images with faces on it, for the images with no faces on them we'll create labels for them manually.

In [14]:
images = os.listdir(dataset_images_path)
for image in images:
    image_name = image.replace(".jpg", "")
    label_path = os.path.join(dataset_labels_path, f"{image_name}.txt")
    if not os.path.exists(label_path):
        with open(label_path, "w") as file:
            file.write("0, 0, 0, 0, 0")

One final step before launching training, we have to create a `dataset.yaml` file inside the yolov5 folder.

In [34]:
classes = None
with open(os.path.join(dataset_labels_path, "classes.txt"), "r") as file:
    classes = list(map(lambda classname: classname.replace("\n", ""), file.readlines()))

with open(os.path.join("yolov5", "dataset.yaml"), "w") as file:
    file.write("path: ../dataset\n")
    file.write("train: images\n")
    file.write("val: images\n")
    file.write(f"nc: {len(classes)}\n")
    file.write(f"names: {classes}\n")

## Model Re-training

To start training, we run the following command.

In [59]:
!cd yolov5 && python train.py --img 320 --batch 16 --epochs 100 --data dataset.yaml --weights yolov5s.pt --workers 2

^C


## Inference

To start using the model, we'll have to load the model as a custom model so we could load the learned weights.

In [60]:
weights_path = os.path.join("yolov5", "runs", "train")
weights = os.listdir(weights_path)

In [61]:
optimized_weights_path = os.path.join(weights_path, weights[-1], 'weights', 'last.pt')
model = torch.hub.load('ultralytics/yolov5', 'custom', path=optimized_weights_path, force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\fouadhanani/.cache\torch\hub\master.zip
YOLOv5  2022-8-16 Python-3.8.10 torch-1.12.1+cpu CPU

Fusing layers... 
Model summary: 213 layers, 7053277 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


In [66]:
# img = "https://img.freepik.com/photos-premium/voyageur-vietnamien-deux-hommes-prenant-selfie-dans-ville-etrangere_264197-5448.jpg"
img = "https://static.techspot.com/images2/news/bigimage/2020/06/2020-06-08-image-8-j_1100.webp"
results = model(img)
results.show()

In [65]:
capture = cv2.VideoCapture(0)

while True:
    status, frame = capture.read()

    results = model(frame)

    cv2.imshow("frame", np.squeeze(results.render()))

    key = cv2.waitKey(1)
    
    # if the "q" key is pressed then stop
    if key & 0xFF == ord('q'):
        break
    
capture.release()
cv2.destroyAllWindows()